Here we I will show how to analyse the primordial feature chains

# Importing libraries

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from postprocessing import *
import os

The main object we will be using to analyse the chains is the posterior binned along the frequency axis. I will start specifying a few parameters:

In [ ]:
chain,logprob = get_total_chain('PrimFeature_lin_tutorial_range2',2,0.5,10)

We save the combined chain:

In [ ]:
with h5.File('chains/PrimFeature_lin_tutorial_range2_total.h5', 'w') as out_f:
    save_dataset(out_f,'chain',chain)
    save_dataset(out_f,'logprob',logprob)

Let us take a look at the posterior for a specific freq. range:

In [2]:
handle_list = ['chains/PrimFeature_lin_tutorial_range1_total.h5','chains/PrimFeature_lin_tutorial_range2_total.h5']
range_limits = [[100,400],[400,650]]
file_output = 'test.h5'
dir_out = ''
param_map = {'omega':3, 'amplitude':2}
omega_bin = 10

In [3]:
GetBinnedPosterior_1D(handle_list,range_limits,file_output,dir_out, param_map,verbose = False,omega_bin = omega_bin)

Problem loading the file chains/PrimFeature_lin_tutorial_range1_total.h5: [Errno 2] Unable to open file (unable to open file: name = 'chains/PrimFeature_lin_tutorial_range1_total.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'chains/PrimFeature_lin_tutorial_range1_total.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
with h5.File('binned_posterior_unit0_lin_data_no_winfunc_unit0_omegalin_9000_dk_0.0005.h5','r') as f:
    nw_1sigma = f['A_cred_1sigma'][:]
    nw_2sigma = f['A_cred_2sigma'][:]
    nw_3sigma = f['A_cred_3sigma'][:]

In [ ]:
omega_min, omega_max = 8500, 9500
omega_bins = np.arange(omega_min, omega_max + omega_bin, omega_bin)
omega_ctrs = 0.5 * (omega_bins[1:] + omega_bins[:-1])

In [ ]:
plt.figure()
plt.plot(omega_ctrs,DESI_1sigma, label = "credible interval ($1\sigma$)")
plt.plot(omega_ctrs,eBOSS_1sigma, label = "credible interval ($1\sigma$)")
plt.legend()
plt.xlabel(r'$\omega_{\rm lin}$')
plt.ylabel(r'$P(|A|)<= threshold$')

In [ ]:
with h5.File('binned_posterior_total_lin_range1_pk0_QSO_mock_5_dk_0.001_kmin_0.005_kmax_0.22.h5','r') as f:
    mock5_1sigma = f['A_cred_1sigma'][:]
    mock5_2sigma = f['A_cred_2sigma'][:]
    mock5_3sigma = f['A_cred_3sigma'][:]

In [ ]:
#mock1 = h5.File('binned_posterior_total_lin_range1_pk0_QSO_mock_1_dk_0.001_kmin_0.005_kmax_0.22.h5')
mock2 = h5.File('binned_posterior_total_lin_range1_pk0_QSO_mock_2_dk_0.001_kmin_0.005_kmax_0.22.h5')
mock3 = h5.File('binned_posterior_total_lin_range1_pk0_QSO_mock_3_dk_0.001_kmin_0.005_kmax_0.22.h5')
mock4 = h5.File('binned_posterior_total_lin_range1_pk0_QSO_mock_4_dk_0.001_kmin_0.005_kmax_0.22.h5')
mock5 = h5.File('binned_posterior_total_lin_range1_pk0_QSO_mock_5_dk_0.001_kmin_0.005_kmax_0.22.h5')

In [ ]:
plt.plot(mock1['posterior:[180, 190]'][:][0],mock1['posterior:[180, 190]'][:][1])
plt.plot(mock1['posterior:[150, 160]'][:][0],mock1['posterior:[150, 160]'][:][1])
plt.plot(mock1['posterior:[200, 210]'][:][0],mock1['posterior:[200, 210]'][:][1])
plt.xlim(-0.5,0.5)